Single trade each day version, 1.25MW battery (nominal charge 20% or 0.25 MW)

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")
dat1=pd.DataFrame(dat)
dat1=dat1.iloc[152:,:].reset_index(drop=True)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]


def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]

        max_price_index = current_Q_A['Price'].idxmax()
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index)]
        min_price_index = current_Q_B['Price'].idxmin()
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > min_price_index)]

        min_price_index1 = None
        max_price_index1 = None

        if len(remaining_prices_0_B) > 0:
            min_price_index1 = remaining_prices_0_B['Price'].idxmin()
        if len(remaining_prices_0_A) > 0:
            max_price_index1 = remaining_prices_0_A['Price'].idxmax()

        if max_price_index is not None and min_price_index1 is not None and max_price_index1 is not None:
            if (current_Q_A.loc[max_price_index, 'Price'] - current_Q_B.loc[min_price_index1, 'Price']) > (current_Q_A.loc[max_price_index1, 'Price'] - current_Q_B.loc[min_price_index, 'Price']):
                chosen_max_price_index = max_price_index
                chosen_min_price_index = min_price_index1
            else:
                chosen_max_price_index = max_price_index1
                chosen_min_price_index = min_price_index
        elif max_price_index is not None and min_price_index1 is not None:
            chosen_max_price_index = max_price_index
            chosen_min_price_index = min_price_index1
        elif max_price_index1 is not None and min_price_index is not None:
            chosen_max_price_index = max_price_index1
            chosen_min_price_index = min_price_index

        if chosen_max_price_index in current_df.index and chosen_min_price_index in current_df.index:
            profit = ((current_df.loc[chosen_max_price_index, 'Price']) * eff_1) - ((current_df.loc[chosen_min_price_index, 'Price']) / eff_2)
            prices.append((chosen_min_price_index, current_df.loc[chosen_min_price_index, 'Price'], chosen_max_price_index, current_df.loc[chosen_max_price_index, 'Price'], profit))

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit'])


r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_dam_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)

PF_DAM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=1, eff_2=1)

print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))

/home/ciaran/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[16733.87]
[16635.88]
[16978.17]
[16716.54]
[15535.64]
[16966.1]
[16642.67]
[26900.24]


In [2]:
r_dam_50_50

,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit
0,5,12.26,17,31.66,12.817796
1,28,12.89,42,36.00,15.646939
2,50,16.23,65,37.16,13.166776
3,85,15.00,90,37.00,14.293878
4,99,17.40,104,35.19,10.396898
...,...,...,...,...,...
358,8640,88.64,8659,180.29,53.783020
359,8667,86.24,8673,219.04,87.232000
360,8692,87.65,8696,252.37,112.457224
361,8715,77.62,8721,128.52,23.611918


Key aspect with perfect efficency profit equals 26900.24

Multi trade each day version, 1.25MW battery (nominal charge 20% or 0.25 MW)

In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")
dat1=pd.DataFrame(dat)
dat1=dat1.iloc[152:,:].reset_index(drop=True)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]



def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index)]
        min_price_index = current_Q_B['Price'].idxmin()
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > min_price_index)]

        min_price_index1 = None
        max_price_index1 = None

        if len(remaining_prices_0_B) > 0:
            min_price_index1 = remaining_prices_0_B['Price'].idxmin()
        if len(remaining_prices_0_A) > 0:
            max_price_index1 = remaining_prices_0_A['Price'].idxmax()

        if max_price_index is not None and min_price_index1 is not None and max_price_index1 is not None:
            if (current_Q_A.loc[max_price_index, 'Price'] - current_Q_B.loc[min_price_index1, 'Price']) > (current_Q_A.loc[max_price_index1, 'Price'] - current_Q_B.loc[min_price_index, 'Price']):
                chosen_max_price_index = max_price_index
                chosen_min_price_index = min_price_index1
            else:
                chosen_max_price_index = max_price_index1
                chosen_min_price_index = min_price_index
        elif max_price_index is not None and min_price_index1 is not None:
            chosen_max_price_index = max_price_index
            chosen_min_price_index = min_price_index1
        elif max_price_index1 is not None and min_price_index is not None:
            chosen_max_price_index = max_price_index1
            chosen_min_price_index = min_price_index

        if chosen_max_price_index in current_df.index and chosen_min_price_index in current_df.index:
            profit = ((current_df.loc[chosen_max_price_index, 'Price']) * eff_1) - ((current_df.loc[chosen_min_price_index, 'Price']) / eff_2)
            prices.append((chosen_min_price_index, current_df.loc[chosen_min_price_index, 'Price'], chosen_max_price_index, current_df.loc[chosen_max_price_index, 'Price'], profit))



            
            
            
            
            
            
        current_df_before_min = current_df[current_df.index < chosen_min_price_index]
        current_Q_A_before_min =  Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < chosen_min_price_index)]
        max_price_index_before_min = None
        if not current_Q_A_before_min.empty:
            max_price_index_before_min = current_Q_A_before_min['Price'].idxmax()
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index_before_min)]
        current_Q_B_before_min =  Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < chosen_min_price_index)]
        min_price_index_before_min = None
        if not current_Q_B_before_min.empty:
            min_price_index_before_min = current_Q_B_before_min['Price'].idxmin()
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > min_price_index_before_min) & (Q_A_Preds.index < chosen_min_price_index)]

        min_price_index2 = None
        max_price_index2 = None

        if len(remaining_prices_1_B) > 0:
            min_price_index2 = remaining_prices_1_B['Price'].idxmin()
        if len(remaining_prices_1_A) > 0:
            max_price_index2 = remaining_prices_1_A['Price'].idxmax()
            
        chosen_max_price_index1 = None
        chosen_min_price_index1 = None
        if max_price_index_before_min is not None and min_price_index2 is not None and max_price_index2 is not None:
            if (current_Q_A.loc[max_price_index_before_min, 'Price'] - current_Q_B.loc[min_price_index2, 'Price']) > (current_Q_A.loc[max_price_index2, 'Price'] - current_Q_B.loc[min_price_index_before_min, 'Price']):
                chosen_max_price_index1 = max_price_index_before_min
                chosen_min_price_index1 = min_price_index2
            else:
                chosen_max_price_index1 = max_price_index2
                chosen_min_price_index1 = min_price_index_before_min
        elif max_price_index_before_min is not None and min_price_index2 is not None:
            chosen_max_price_index1 = max_price_index_before_min
            chosen_min_price_index1 = min_price_index2
        elif max_price_index2 is not None and min_price_index_before_min is not None:
            chosen_max_price_index1 = max_price_index2
            chosen_min_price_index1 = min_price_index_before_min

        if chosen_max_price_index1 in current_df.index and chosen_min_price_index1 in current_df.index:
            profit = ((current_df.loc[chosen_max_price_index1, 'Price']) * eff_1) - ((current_df.loc[chosen_min_price_index1, 'Price']) / eff_2)
            prices.append((chosen_min_price_index1, current_df.loc[chosen_min_price_index1, 'Price'], chosen_max_price_index1, current_df.loc[chosen_max_price_index1, 'Price'], profit))
            
              
            

        current_df_after_max = current_df[current_df.index > chosen_max_price_index]
        current_Q_A_after_max =  Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > chosen_max_price_index)]
        max_price_index_after_max = None
        if not current_Q_A_after_max.empty:
            max_price_index_after_max = current_Q_A_after_max['Price'].idxmax()
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index_after_max)& (Q_B_Preds.index > chosen_max_price_index)]
        
        current_Q_B_after_max =  Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > chosen_max_price_index)]
        min_price_index_after_max = None
        if not current_Q_B_after_max.empty:
            min_price_index_after_max = current_Q_B_after_max['Price'].idxmin()
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > min_price_index_after_max)]

        min_price_index3 = None
        max_price_index3 = None

        if len(remaining_prices_2_B) > 0:
            min_price_index3 = remaining_prices_2_B['Price'].idxmin()
        if len(remaining_prices_2_A) > 0:
            max_price_index3 = remaining_prices_2_A['Price'].idxmax()
            
        chosen_max_price_index2 = None
        chosen_min_price_index2 = None  
        if max_price_index_after_max is not None and min_price_index3 is not None and max_price_index3 is not None:
            if (current_Q_A.loc[max_price_index_after_max, 'Price'] - current_Q_B.loc[min_price_index3, 'Price']) > (current_Q_A.loc[max_price_index3, 'Price'] - current_Q_B.loc[min_price_index_after_max, 'Price']):
                chosen_max_price_index2 = max_price_index_after_max
                chosen_min_price_index2 = min_price_index3
            else:
                chosen_max_price_index2 = max_price_index3
                chosen_min_price_index2 = min_price_index_after_max
        elif max_price_index_after_max is not None and min_price_index3 is not None:
            chosen_max_price_index2 = max_price_index_after_max
            chosen_min_price_index2 = min_price_index3
        elif max_price_index3 is not None and min_price_index_after_max is not None:
            chosen_max_price_index2 = max_price_index3
            chosen_min_price_index2 = min_price_index_after_max

        if chosen_max_price_index2 in current_df.index and chosen_min_price_index2 in current_df.index:
            profit = ((current_df.loc[chosen_max_price_index2, 'Price']) * eff_1) - ((current_df.loc[chosen_min_price_index2, 'Price']) / eff_2)
            prices.append((chosen_min_price_index2, current_df.loc[chosen_min_price_index2, 'Price'], chosen_max_price_index2, current_df.loc[chosen_max_price_index2, 'Price'], profit))

            
            
    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit'])

            
            
            
            

r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_dam_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
PF_DAM     =run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r , Q_B_Preds=Y_r,  eff_1=1, eff_2=1)

print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))

[15444.07]
[15939.93]
[15869.64]
[14866.83]
[14024.75]
[15553.38]
[15562.84]
[28276.46]


In [4]:
r_dam_50_50.head(n=20)

,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit
0,5,12.26,17,31.66,12.817796
1,28,12.89,42,36.00,15.646939
2,26,12.88,27,10.26,-4.934857
3,50,16.23,65,37.16,13.166776
4,85,15.00,90,37.00,14.293878
5,75,17.99,80,29.42,5.178857
6,99,17.40,104,35.19,10.396898
7,106,31.90,107,32.46,-6.583020
8,121,-0.05,137,21.49,17.243020
9,158,-27.93,163,12.78,38.724000


key aspect with perfect efficiency profit equals 28276.46, greater than single trades 26900.24. Yet the profit for all trades with battery paramaters are poor.

Bottleneck strategy, flexible timestamp order

In [5]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_DAM_1-12.csv")
dat1=pd.DataFrame(dat)
dat1=dat1.iloc[152:,:].reset_index(drop=True)


column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]


def process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index):
    if min_price_index < max_price_index:
        bottleneck_1 = min(capacity - charge_level, ramp_rate)
        charge_level += bottleneck_1
        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
        charge_level -= bottleneck_2
        profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1) / eff_2)
        prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
    elif min_price_index > max_price_index:
        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
        charge_level -= bottleneck_2
        bottleneck_1 = min(capacity - charge_level, ramp_rate)
        charge_level += bottleneck_1
        profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1) / eff_2)
        prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))        
    return charge_level

def recursive_process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, remaining_prices_A, remaining_prices_B, current_Q_A, current_Q_B, level_0):
    if len(remaining_prices_A) <= 1:
        return charge_level

    max_price_index = remaining_prices_A['Price'].idxmax()
    min_price_index = remaining_prices_B['Price'].idxmin()

    if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
        charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index)

    smaller_index = min(min_price_index, max_price_index)
    larger_index = max(min_price_index, max_price_index)

    remaining_prices_A = current_Q_A[(current_Q_A['level_0'] == level_0) & (current_Q_A.index > smaller_index) & (current_Q_A.index < larger_index)]
    remaining_prices_B = current_Q_B[(current_Q_B['level_0'] == level_0) & (current_Q_B.index > smaller_index) & (current_Q_B.index < larger_index)]

    charge_level = recursive_process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, remaining_prices_A, remaining_prices_B, current_Q_A, current_Q_B, level_0)

    return charge_level

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]

        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]

        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]
        
        if len(remaining_prices_2) > 1:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_3, 'Price'] < current_Q_A.loc[max_price_index_3, 'Price']:
                    charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index_3, max_price_index_3)            
                else:
                    continue     
                    
                    
                                    
        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index)            
        else:
            continue    
                                   
        if len(remaining_prices_1) > 1:
            charge_level = recursive_process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, remaining_prices_1_A, remaining_prices_1_B, current_Q_A, current_Q_B, level_0)
        else:
            continue     
            
            
                                                                                    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index_1, max_price_index_1)            
            else:
                continue                                
                                    
                                            
    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
r_dam_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
r_dam_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
r_dam_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
r_dam_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)

PF_DAM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)

print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))

[32897.74]
[19125.62]
[26440.41]
[25507.78]
[24230.32]
[18355.34]
[14222.23]
[34703.48]


Single Trade best performer: 16978.17, Multi Trade: 15939.93, & Bottleneck strategy: 32897.74.  nearly twice that of single trade.

BM Market performance for each strategy.

Single Trade BM:

In [6]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat1 = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
dat1=dat1.iloc[456:,:].reset_index(drop=True)

column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]


def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]

        max_price_index = current_Q_A['Price'].idxmax()
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index)]
        min_price_index = current_Q_B['Price'].idxmin()
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > min_price_index)]

        min_price_index1 = None
        max_price_index1 = None

        if len(remaining_prices_0_B) > 0:
            min_price_index1 = remaining_prices_0_B['Price'].idxmin()
        if len(remaining_prices_0_A) > 0:
            max_price_index1 = remaining_prices_0_A['Price'].idxmax()

        if max_price_index is not None and min_price_index1 is not None and max_price_index1 is not None:
            if (current_Q_A.loc[max_price_index, 'Price'] - current_Q_B.loc[min_price_index1, 'Price']) > (current_Q_A.loc[max_price_index1, 'Price'] - current_Q_B.loc[min_price_index, 'Price']):
                chosen_max_price_index = max_price_index
                chosen_min_price_index = min_price_index1
            else:
                chosen_max_price_index = max_price_index1
                chosen_min_price_index = min_price_index
        elif max_price_index is not None and min_price_index1 is not None:
            chosen_max_price_index = max_price_index
            chosen_min_price_index = min_price_index1
        elif max_price_index1 is not None and min_price_index is not None:
            chosen_max_price_index = max_price_index1
            chosen_min_price_index = min_price_index

        if chosen_max_price_index in current_df.index and chosen_min_price_index in current_df.index:
            profit = ((current_df.loc[chosen_max_price_index, 'Price']) * eff_1) - ((current_df.loc[chosen_min_price_index, 'Price']) / eff_2)
            prices.append((chosen_min_price_index, current_df.loc[chosen_min_price_index, 'Price'], chosen_max_price_index, current_df.loc[chosen_max_price_index, 'Price'], profit))

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit'])



r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_bm_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_bm_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_bm_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_bm_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)

PF_BM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=1, eff_2=1)

print(np.round(sum(r_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_BM.iloc[:,4:5].values),2))

[4413.06]
[3336.58]
[6189.9]
[5816.05]
[7045.12]
[6611.12]
[3822.13]
[110250.32]


In [7]:
r_bm_50_50 

,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit
0,13,33.28,15,60.90,14.760816
1,16,64.04,22,45.66,-28.818939
2,35,8.05,47,27.74,13.977714
3,58,33.00,63,10.61,-25.185469
4,73,17.02,76,84.08,49.896653
...,...,...,...,...,...
1017,17440,84.82,17446,236.47,102.624980
1018,17462,65.43,17471,59.32,-19.309306
1019,17483,-28.14,17487,48.61,67.602286
1020,17488,46.40,17494,71.75,10.053061


Multi Trade BM:

In [8]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat1 = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
dat1=dat1.iloc[456:,:].reset_index(drop=True)

column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
#First pair              
        max_price_index = current_Q_A['Price'].idxmax()
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index)]
        min_price_index = current_Q_B['Price'].idxmin()
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > min_price_index)]

        min_price_index1 = None
        max_price_index1 = None

        if len(remaining_prices_0_B) > 0:
            min_price_index1 = remaining_prices_0_B['Price'].idxmin()
        if len(remaining_prices_0_A) > 0:
            max_price_index1 = remaining_prices_0_A['Price'].idxmax()

        if max_price_index is not None and min_price_index1 is not None and max_price_index1 is not None:
            if (current_Q_A.loc[max_price_index, 'Price'] - current_Q_B.loc[min_price_index1, 'Price']) > (current_Q_A.loc[max_price_index1, 'Price'] - current_Q_B.loc[min_price_index, 'Price']):
                chosen_max_price_index = max_price_index
                chosen_min_price_index = min_price_index1
            else:
                chosen_max_price_index = max_price_index1
                chosen_min_price_index = min_price_index
        elif max_price_index is not None and min_price_index1 is not None:
            chosen_max_price_index = max_price_index
            chosen_min_price_index = min_price_index1
        elif max_price_index1 is not None and min_price_index is not None:
            chosen_max_price_index = max_price_index1
            chosen_min_price_index = min_price_index

        if chosen_max_price_index in current_df.index and chosen_min_price_index in current_df.index:
            profit = ((current_df.loc[chosen_max_price_index, 'Price']) * eff_1) - ((current_df.loc[chosen_min_price_index, 'Price']) / eff_2)
            prices.append((chosen_min_price_index, current_df.loc[chosen_min_price_index, 'Price'], chosen_max_price_index, current_df.loc[chosen_max_price_index, 'Price'], profit))

#first subset              

            
        current_df_before_min = current_df[current_df.index < chosen_min_price_index]
        current_Q_A_before_min =  Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < chosen_min_price_index)]
        max_price_index_before_min = None
        if not current_Q_A_before_min.empty:
            max_price_index_before_min = current_Q_A_before_min['Price'].idxmax()
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index_before_min)]
        current_Q_B_before_min =  Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < chosen_min_price_index)]
        min_price_index_before_min = None
        if not current_Q_B_before_min.empty:
            min_price_index_before_min = current_Q_B_before_min['Price'].idxmin()
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > min_price_index_before_min) & (Q_A_Preds.index < chosen_min_price_index)]

        min_price_index2 = None
        max_price_index2 = None

        if len(remaining_prices_1_B) > 0:
            min_price_index2 = remaining_prices_1_B['Price'].idxmin()
        if len(remaining_prices_1_A) > 0:
            max_price_index2 = remaining_prices_1_A['Price'].idxmax()
            
        chosen_max_price_index1 = None
        chosen_min_price_index1 = None
        if max_price_index_before_min is not None and min_price_index2 is not None and max_price_index2 is not None:
            if (current_Q_A.loc[max_price_index_before_min, 'Price'] - current_Q_B.loc[min_price_index2, 'Price']) > (current_Q_A.loc[max_price_index2, 'Price'] - current_Q_B.loc[min_price_index_before_min, 'Price']):
                chosen_max_price_index1 = max_price_index_before_min
                chosen_min_price_index1 = min_price_index2
            else:
                chosen_max_price_index1 = max_price_index2
                chosen_min_price_index1 = min_price_index_before_min
        elif max_price_index_before_min is not None and min_price_index2 is not None:
            chosen_max_price_index1 = max_price_index_before_min
            chosen_min_price_index1 = min_price_index2
        elif max_price_index2 is not None and min_price_index_before_min is not None:
            chosen_max_price_index1 = max_price_index2
            chosen_min_price_index1 = min_price_index_before_min

        if chosen_max_price_index1 in current_df.index and chosen_min_price_index1 in current_df.index:
            profit = ((current_df.loc[chosen_max_price_index1, 'Price']) * eff_1) - ((current_df.loc[chosen_min_price_index1, 'Price']) / eff_2)
            prices.append((chosen_min_price_index1, current_df.loc[chosen_min_price_index1, 'Price'], chosen_max_price_index1, current_df.loc[chosen_max_price_index1, 'Price'], profit))
            
#second subset              
            

        current_df_after_max = current_df[current_df.index > chosen_max_price_index]
        current_Q_A_after_max =  Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > chosen_max_price_index)]
        max_price_index_after_max = None
        if not current_Q_A_after_max.empty:
            max_price_index_after_max = current_Q_A_after_max['Price'].idxmax()
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index_after_max)& (Q_B_Preds.index > chosen_max_price_index)]
        
        current_Q_B_after_max =  Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > chosen_max_price_index)]
        min_price_index_after_max = None
        if not current_Q_B_after_max.empty:
            min_price_index_after_max = current_Q_B_after_max['Price'].idxmin()
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > min_price_index_after_max)]

        min_price_index3 = None
        max_price_index3 = None

        if len(remaining_prices_2_B) > 0:
            min_price_index3 = remaining_prices_2_B['Price'].idxmin()
        if len(remaining_prices_2_A) > 0:
            max_price_index3 = remaining_prices_2_A['Price'].idxmax()
            
        chosen_max_price_index2 = None
        chosen_min_price_index2 = None  
        if max_price_index_after_max is not None and min_price_index3 is not None and max_price_index3 is not None:
            if (current_Q_A.loc[max_price_index_after_max, 'Price'] - current_Q_B.loc[min_price_index3, 'Price']) > (current_Q_A.loc[max_price_index3, 'Price'] - current_Q_B.loc[min_price_index_after_max, 'Price']):
                chosen_max_price_index2 = max_price_index_after_max
                chosen_min_price_index2 = min_price_index3
            else:
                chosen_max_price_index2 = max_price_index3
                chosen_min_price_index2 = min_price_index_after_max
        elif max_price_index_after_max is not None and min_price_index3 is not None:
            chosen_max_price_index2 = max_price_index_after_max
            chosen_min_price_index2 = min_price_index3
        elif max_price_index3 is not None and min_price_index_after_max is not None:
            chosen_max_price_index2 = max_price_index3
            chosen_min_price_index2 = min_price_index_after_max

        if chosen_max_price_index2 in current_df.index and chosen_min_price_index2 in current_df.index:
            profit = ((current_df.loc[chosen_max_price_index2, 'Price']) * eff_1) - ((current_df.loc[chosen_min_price_index2, 'Price']) / eff_2)
            prices.append((chosen_min_price_index2, current_df.loc[chosen_min_price_index2, 'Price'], chosen_max_price_index2, current_df.loc[chosen_max_price_index2, 'Price'], profit))

            
            
    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit'])


r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_bm_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_bm_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_bm_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_bm_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)

PF_BM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=1, eff_2=1)

print(np.round(sum(r_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_BM.iloc[:,4:5].values),2))

[421.91]
[-3163.36]
[1017.54]
[1119.05]
[1226.63]
[1416.71]
[-2832.35]
[147648.52]


In [9]:
r_bm_50_50 

,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit
0,13,33.28,15,60.90,14.760816
1,11,16.99,12,55.52,27.079265
2,16,64.04,22,45.66,-28.818939
3,23,47.94,25,44.76,-13.110367
4,35,8.05,47,27.74,13.977714
...,...,...,...,...,...
1445,17483,-28.14,17487,48.61,67.602286
1446,17472,63.91,17473,59.17,-17.878286
1447,17488,46.40,17494,71.75,10.053061
1448,17497,71.11,17499,58.45,-25.801224


Bottleneck strategy BM:

In [10]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat1 = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
dat1=dat1.iloc[456:,:].reset_index(drop=True)

column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index):
    if min_price_index < max_price_index:
        bottleneck_1 = min(capacity - charge_level, ramp_rate)
        charge_level += bottleneck_1
        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
        charge_level -= bottleneck_2
        profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1) / eff_2)
        prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
    elif min_price_index > max_price_index:
        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
        charge_level -= bottleneck_2
        bottleneck_1 = min(capacity - charge_level, ramp_rate)
        charge_level += bottleneck_1
        profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1) / eff_2)
        prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))        
    return charge_level

def recursive_process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, remaining_prices_A, remaining_prices_B, current_Q_A, current_Q_B, level_0):
    if len(remaining_prices_A) <= 1:
        return charge_level

    max_price_index = remaining_prices_A['Price'].idxmax()
    min_price_index = remaining_prices_B['Price'].idxmin()

    if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
        charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index)

    smaller_index = min(min_price_index, max_price_index)
    larger_index = max(min_price_index, max_price_index)

    remaining_prices_A = current_Q_A[(current_Q_A['level_0'] == level_0) & (current_Q_A.index > smaller_index) & (current_Q_A.index < larger_index)]
    remaining_prices_B = current_Q_B[(current_Q_B['level_0'] == level_0) & (current_Q_B.index > smaller_index) & (current_Q_B.index < larger_index)]

    charge_level = recursive_process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, remaining_prices_A, remaining_prices_B, current_Q_A, current_Q_B, level_0)

    return charge_level

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]

        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]

        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]
        
        if len(remaining_prices_2) > 1:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_3, 'Price'] < current_Q_A.loc[max_price_index_3, 'Price']:
                    charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index_3, max_price_index_3)            
                else:
                    continue     
                    
                    
                                    
        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index)            
        else:
            continue    
                                   
        if len(remaining_prices_1) > 1:
            charge_level = recursive_process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, remaining_prices_1_A, remaining_prices_1_B, current_Q_A, current_Q_B, level_0)
        else:
            continue     
            
            
                                                                                    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index_1, max_price_index_1)            
            else:
                continue                                
                                    
                                            
    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
r_bm_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
r_bm_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
r_bm_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
r_bm_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)
PF_BM     =run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r,  Q_B_Preds=Y_r,  eff_1=1, eff_2=1, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0)

print(np.round(sum(r_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_BM.iloc[:,4:5].values),2))

[25163.64]
[17566.95]
[26770.61]
[27042.28]
[20641.57]
[13562.66]
[-215.86]
[217326.57]


In [11]:
r_bm_50_50  

,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,13,33.28,0,187.63,-33.959184,1
1,11,16.99,5,100.34,62.935265,1
2,9,175.37,7,46.23,-141.964980,1
3,14,28.95,15,60.90,48.720000,0
4,16,64.04,19,52.40,-23.426939,0
...,...,...,...,...,...,...
5810,17507,36.82,17516,57.13,8.132571,0
5811,17508,39.54,17515,68.65,14.573061,0
5812,17509,40.81,17514,61.67,7.693143,0
5813,17510,53.13,17513,64.99,-2.222286,0


Single Trade BM best result: 7045.12, Multi Trade BM best result: 1416.71 , & bottleneck strategy Trade BM best result: 27042.28. Multi trade struggles despite more trades, effieiency hampers strategy. Bottleneck is still best.

Single Trade BM perfect efficiency: 110250.32, Multi Trade BM perfect efficiency: 147648.52 , & bottleneck strategy Trade BM perfect efficiency: 217514.2. With perfect foresight Multi trade is better than single trade despite worse results above. Bottleneck is still best.

Lastly Dual market trading.

In [13]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

num_rows = 365 * 48
df = pd.DataFrame(index=range(num_rows))

# Add the 'level_0' column
df['level_0'] = np.repeat(np.arange(366), 48)[:num_rows]

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)

dat1 = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
dat1=dat1.iloc[456:,:].reset_index(drop=True)
dat1=pd.concat([dat1, df],axis=1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r_BM=dat1[column_names].dropna().stack().reset_index()
Y_r_BM["Price"]=Y_r_BM.iloc[:,2:3]
Q_10_BM["Price"]=Q_10_BM.iloc[:,2:3]
Q_30_BM["Price"]=Q_30_BM.iloc[:,2:3]
Q_50_BM["Price"]=Q_50_BM.iloc[:,2:3]
Q_70_BM["Price"]=Q_70_BM.iloc[:,2:3]
Q_90_BM["Price"]=Q_90_BM.iloc[:,2:3]

Y_r_BM  = Y_r_BM.iloc[:,1:]
Q_10_BM = Q_10_BM.iloc[:,1:]
Q_30_BM = Q_30_BM.iloc[:,1:]
Q_50_BM = Q_50_BM.iloc[:,1:]
Q_70_BM = Q_70_BM.iloc[:,1:]
Q_90_BM = Q_90_BM.iloc[:,1:]

Y_r_BM= pd.concat([Y_r_BM, df],axis=1)
Q_10_BM= pd.concat([Q_10_BM, df],axis=1)
Q_30_BM= pd.concat([Q_30_BM, df],axis=1)
Q_50_BM= pd.concat([Q_50_BM, df],axis=1)
Q_70_BM= pd.concat([Q_70_BM, df],axis=1)
Q_90_BM= pd.concat([Q_90_BM, df],axis=1)

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_DAM_1-12.csv")
dat1=pd.DataFrame(dat)
dat1=dat1.iloc[152:,:].reset_index(drop=True)
start = 0
end = 17519
step = 2

values = list(range(start, end+1, step))
df1 = pd.DataFrame({'P_dam': values})

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r_DAM=dat1[column_names].dropna().stack().reset_index()
Y_r_DAM=Y_r_DAM.iloc[:,:]

Y_r_DAM["Price"]=Y_r_DAM.iloc[:,2:3]
Q_10_DAM["Price"]=Q_10_DAM.iloc[:,2:3]
Q_30_DAM["Price"]=Q_30_DAM.iloc[:,2:3]
Q_50_DAM["Price"]=Q_50_DAM.iloc[:,2:3]
Q_70_DAM["Price"]=Q_70_DAM.iloc[:,2:3]
Q_90_DAM["Price"]=Q_90_DAM.iloc[:,2:3]

Y_r_DAM= pd.concat([Y_r_DAM, df1], axis=1)
Q_10_DAM= pd.concat([Q_10_DAM, df1], axis=1)
Q_30_DAM= pd.concat([Q_30_DAM, df1], axis=1)
Q_50_DAM= pd.concat([Q_50_DAM, df1], axis=1)
Q_70_DAM= pd.concat([Q_70_DAM, df1], axis=1)
Q_90_DAM= pd.concat([Q_90_DAM, df1], axis=1)


def run_electricity_strategy(df, df_bm, Q_A_Preds, Q_B_Preds, Q_A_Preds_bm, Q_B_Preds_bm,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        current_df_bm = df_bm[df_bm['level_0'] == level_0]
        current_Q_A_bm = Q_A_Preds_bm[Q_A_Preds_bm['level_0'] == level_0]
        current_Q_B_bm = Q_B_Preds_bm[Q_B_Preds_bm['level_0'] == level_0]  
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()
        
        max_price_index = current_Q_A.loc[max_price_index, 'P_dam']
        min_price_index = current_Q_B.loc[min_price_index, 'P_dam']        

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        
        remaining_prices_0 = current_df[(current_df['P_dam'] > smaller_index) & (current_df['P_dam'] < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index) & (Q_A_Preds['P_dam'] < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index) & (Q_B_Preds['P_dam'] < larger_index)]
        
        remaining_prices_1 = current_df_bm[(current_df_bm.index > smaller_index) & (current_df_bm.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index) & (Q_A_Preds_bm.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index) & (Q_B_Preds_bm.index > larger_index)]
        remaining_prices_2 = current_df_bm[(current_df_bm.index < smaller_index) & (current_df_bm.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index < smaller_index) & (Q_A_Preds_bm.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index < smaller_index) & (Q_B_Preds_bm.index < larger_index)]

        if current_Q_B.loc[min_price_index/2, 'Price'] < current_Q_A.loc[max_price_index/2, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'], max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'],  max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                               
                        
        if len(remaining_prices_0) > 1:
            max_price_index_0 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_0 = remaining_prices_0_B['Price'].idxmin()            
            max_price_index_0 = remaining_prices_0_A.loc[max_price_index_0, 'P_dam']
            min_price_index_0 = remaining_prices_0_B.loc[min_price_index_0, 'P_dam']
            
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_0/2, 'Price'] < current_Q_A.loc[max_price_index_0/2, 'Price']:
                  if min_price_index_0 < max_price_index_0:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_0/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0/2, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_0, current_df.loc[min_price_index_0/2, 'Price'], max_price_index_0, current_df.loc[max_price_index_0/2, 'Price'], profit, charge_level))
            
                  elif min_price_index_0 > max_price_index_0:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_0/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0/2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_0, current_df.loc[min_price_index_0/2, 'Price'], max_price_index_0, current_df.loc[max_price_index_0/2, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                

            smaller_index_0a = min(min_price_index_0, max_price_index_0)
            larger_index_0a = max(min_price_index_0, max_price_index_0)
            remaining_prices_0a = current_df[(current_df['P_dam'] > smaller_index_0a) & (current_df['P_dam'] < larger_index_0a)]
            remaining_prices_0a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0a) & (Q_A_Preds['P_dam'] < larger_index_0a)]
            remaining_prices_0a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0a) & (Q_B_Preds['P_dam'] < larger_index_0a)]

            if len(remaining_prices_0a) > 1:

                    max_price_index_0a = remaining_prices_0a_A['Price'].idxmax()
                    min_price_index_0a = remaining_prices_0a_B['Price'].idxmin()
                    max_price_index_0a = remaining_prices_0a_A.loc[max_price_index_0a, 'P_dam']
                    min_price_index_0a = remaining_prices_0a_B.loc[min_price_index_0a, 'P_dam']
                    if current_Q_B.loc[min_price_index_0a/2, 'Price'] < current_Q_A.loc[max_price_index_0a/2, 'Price']:
                        if min_price_index_0a < max_price_index_0a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_0a/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0a/2, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_0a, current_df.loc[min_price_index_0a/2, 'Price'], max_price_index_0a, current_df.loc[max_price_index_0a/2, 'Price'], profit, charge_level))
                    
                        elif min_price_index_0a > max_price_index_0a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_0a/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0a/2, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_0a, current_df.loc[min_price_index_0a/2, 'Price'], max_price_index_0a, current_df.loc[max_price_index_0a/2, 'Price'], profit, charge_level))
                                
                        else:
                             continue     
                                
                                
                                
                                
                    smaller_index_0b = min(min_price_index_0a, max_price_index_0a)
                    larger_index_0b = max(min_price_index_0a, max_price_index_0a)
                    remaining_prices_0b = current_df[(current_df['P_dam'] > smaller_index_0b) & (current_df['P_dam'] < larger_index_0b)]
                    remaining_prices_0b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0b) & (Q_A_Preds['P_dam'] < larger_index_0b)]
                    remaining_prices_0b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0b) & (Q_B_Preds['P_dam'] < larger_index_0b)]

                    if len(remaining_prices_0b) > 1:

                            max_price_index_0b = remaining_prices_0b_A['Price'].idxmax()
                            min_price_index_0b = remaining_prices_0b_B['Price'].idxmin()
                            max_price_index_0b = remaining_prices_0b_A.loc[max_price_index_0b, 'P_dam']
                            min_price_index_0b = remaining_prices_0b_B.loc[min_price_index_0b, 'P_dam']
                            if current_Q_B.loc[min_price_index_0b/2, 'Price'] < current_Q_A.loc[max_price_index_0b/2, 'Price']:
                                if min_price_index_0b < max_price_index_0b:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_0b/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0b/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_0b, current_df.loc[min_price_index_0b/2, 'Price'], max_price_index_0b, current_df.loc[max_price_index_0b/2, 'Price'], profit, charge_level))
                    
                                elif min_price_index_0b > max_price_index_0b:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_0b/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0b/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                      prices.append((min_price_index_0b, current_df.loc[min_price_index_0b/2, 'Price'], max_price_index_0b, current_df.loc[max_price_index_0b/2, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                    
                                
                                
 
                            smaller_index_0c = min(min_price_index_0b, max_price_index_0b)
                            larger_index_0c = max(min_price_index_0b, max_price_index_0b)
                            remaining_prices_0c = current_df[(current_df['P_dam'] > smaller_index_0c) & (current_df['P_dam'] < larger_index_0c)]
                            remaining_prices_0c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0c) & (Q_A_Preds['P_dam'] < larger_index_0c)]
                            remaining_prices_0c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0c) & (Q_B_Preds['P_dam'] < larger_index_0c)]

                            if len(remaining_prices_0c) > 1:

                                    max_price_index_0c = remaining_prices_0c_A['Price'].idxmax()
                                    min_price_index_0c = remaining_prices_0c_B['Price'].idxmin()
                                    max_price_index_0c = remaining_prices_0c_A.loc[max_price_index_0c, 'P_dam']
                                    min_price_index_0c = remaining_prices_0c_B.loc[min_price_index_0c, 'P_dam']
                                    if current_Q_B.loc[min_price_index_0c/2, 'Price'] < current_Q_A.loc[max_price_index_0c/2, 'Price']:
                                        if min_price_index_0c < max_price_index_0c:
                                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                            charge_level += bottleneck_1
                                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                            charge_level -= bottleneck_2
                    
                                            profit = (current_df.loc[max_price_index_0c/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0c/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                            prices.append((min_price_index_0c, current_df.loc[min_price_index_0c/2, 'Price'], max_price_index_0c, current_df.loc[max_price_index_0c/2, 'Price'], profit, charge_level))
                    
                                        elif min_price_index_0c > max_price_index_0c:
                                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                              charge_level -= bottleneck_2
                                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                              charge_level += bottleneck_1
                    
                                              profit = (current_df.loc[max_price_index_0c/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0c/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                              prices.append((min_price_index_0c, current_df.loc[min_price_index_0c/2, 'Price'], max_price_index_0c, current_df.loc[max_price_index_0c/2, 'Price'], profit, charge_level))
                                
                                        else:
                                             continue


                                    smaller_index_0d = min(min_price_index_0c, max_price_index_0c)
                                    larger_index_0d = max(min_price_index_0c, max_price_index_0c)
                                    remaining_prices_0d = current_df[(current_df['P_dam'] > smaller_index_0d) & (current_df['P_dam'] < larger_index_0d)]
                                    remaining_prices_0d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0d) & (Q_A_Preds['P_dam'] < larger_index_0d)]
                                    remaining_prices_0d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0d) & (Q_B_Preds['P_dam'] < larger_index_0d)]

                                    if len(remaining_prices_0d) > 1:

                                            max_price_index_0d = remaining_prices_0d_A['Price'].idxmax()
                                            min_price_index_0d = remaining_prices_0d_B['Price'].idxmin()
                                            max_price_index_0d = remaining_prices_0d_A.loc[max_price_index_0d, 'P_dam']
                                            min_price_index_0d = remaining_prices_0d_B.loc[min_price_index_0d, 'P_dam']
                                            if current_Q_B.loc[min_price_index_0d/2, 'Price'] < current_Q_A.loc[max_price_index_0d/2, 'Price']:
                                                if min_price_index_0d < max_price_index_0d:
                                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                    charge_level += bottleneck_1
                                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                    charge_level -= bottleneck_2
                    
                                                    profit = (current_df.loc[max_price_index_0d/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0d/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                                    prices.append((min_price_index_0d, current_df.loc[min_price_index_0d/2, 'Price'], max_price_index_0d, current_df.loc[max_price_index_0d/2, 'Price'], profit, charge_level))
                    
                                                elif min_price_index_0d > max_price_index_0d:
                                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                      charge_level -= bottleneck_2
                                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                      charge_level += bottleneck_1
                    
                                                      profit = (current_df.loc[max_price_index_0d/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0d/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                                      prices.append((min_price_index_0d, current_df.loc[min_price_index_0d/2, 'Price'], max_price_index_0d, current_df.loc[max_price_index_0d/2, 'Price'], profit, charge_level))
                                
                                                else:
                                                     continue



                                            smaller_index_0e = min(min_price_index_0d, max_price_index_0d)
                                            larger_index_0e = max(min_price_index_0d, max_price_index_0d)
                                            remaining_prices_0e = current_df[(current_df['P_dam'] > smaller_index_0e) & (current_df['P_dam'] < larger_index_0e)]
                                            remaining_prices_0e_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0e) & (Q_A_Preds['P_dam'] < larger_index_0e)]
                                            remaining_prices_0e_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0e) & (Q_B_Preds['P_dam'] < larger_index_0e)]

                                            if len(remaining_prices_0e) > 1:

                                                    max_price_index_0e = remaining_prices_0e_A['Price'].idxmax()
                                                    min_price_index_0e = remaining_prices_0e_B['Price'].idxmin()
                                                    max_price_index_0e = remaining_prices_0e_A.loc[max_price_index_0e, 'P_dam']
                                                    min_price_index_0e = remaining_prices_0e_B.loc[min_price_index_0e, 'P_dam']
                                                    if current_Q_B.loc[min_price_index_0e/2, 'Price'] < current_Q_A.loc[max_price_index_0e/2, 'Price']:
                                                        if min_price_index_0e < max_price_index_0e:
                                                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                            charge_level += bottleneck_1
                                                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                            charge_level -= bottleneck_2
                    
                                                            profit = (current_df.loc[max_price_index_0e/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0e/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                                            prices.append((min_price_index_0e, current_df.loc[min_price_index_0e/2, 'Price'], max_price_index_0e, current_df.loc[max_price_index_0e/2, 'Price'], profit, charge_level))
                    
                                                        elif min_price_index_0e > max_price_index_0e:
                                                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                              charge_level -= bottleneck_2
                                                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                              charge_level += bottleneck_1
                    
                                                              profit = (current_df.loc[max_price_index_0e/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0e/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                                              prices.append((min_price_index_0e, current_df.loc[min_price_index_0e/2, 'Price'], max_price_index_0e, current_df.loc[max_price_index_0e/2, 'Price'], profit, charge_level))
                                
                                                        else:
                                                             continue









                                            
        if len(remaining_prices_1) > 1:

                max_price_index_1 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_1 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_1, 'Price'] < current_Q_A_bm.loc[max_price_index_1, 'Price']:
                      if min_price_index_1 < max_price_index_1:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
            
                            profit = (current_df_bm.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_1, current_df_bm.loc[min_price_index_1, 'Price'], max_price_index_1, current_df_bm.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                      elif min_price_index_1 > max_price_index_1:
                             bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                             charge_level -= bottleneck_2
                             bottleneck_1 = min(capacity - charge_level, ramp_rate)
                             charge_level += bottleneck_1
            
                             profit = (current_df_bm.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                             prices.append((min_price_index_1, current_df_bm.loc[min_price_index_1, 'Price'], max_price_index_1, current_df_bm.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                      else:
                            continue                
                        

        
                smaller_index_1a = min(min_price_index_1, max_price_index_1)
                larger_index_1a = max(min_price_index_1, max_price_index_1)
                remaining_prices_1a = current_df_bm[(current_df_bm.index > smaller_index_1a) & (current_df_bm.index < larger_index_1a)]
                remaining_prices_1a_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1a) & (Q_A_Preds_bm.index < larger_index_1a)]
                remaining_prices_1a_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1a) & (Q_B_Preds_bm.index < larger_index_1a)]

                if len(remaining_prices_1a) > 1:
                        max_price_index_1a = remaining_prices_1a_A['Price'].idxmax()
                        min_price_index_1a = remaining_prices_1a_B['Price'].idxmin()
                        if current_Q_B_bm.loc[min_price_index_1a, 'Price'] < current_Q_A_bm.loc[max_price_index_1a, 'Price']:
                            if min_price_index_1a < max_price_index_1a:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df_bm.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_1a, current_df_bm.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df_bm.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1a > max_price_index_1a:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df_bm.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                                  prices.append((min_price_index_1a, current_df_bm.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df_bm.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                         
                        
                        
                        smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                        larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                        remaining_prices_1b = current_df_bm[(current_df_bm.index > smaller_index_1b) & (current_df_bm.index < larger_index_1b)]
                        remaining_prices_1b_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1b) & (Q_A_Preds_bm.index < larger_index_1b)]
                        remaining_prices_1b_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1b) & (Q_B_Preds_bm.index < larger_index_1b)]

                        if len(remaining_prices_1b) > 1:
                                max_price_index_1b = remaining_prices_1b_A['Price'].idxmax()
                                min_price_index_1b = remaining_prices_1b_B['Price'].idxmin()
                                if current_Q_B_bm.loc[min_price_index_1b, 'Price'] < current_Q_A_bm.loc[max_price_index_1b, 'Price']:
                                    if min_price_index_1b < max_price_index_1b:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df_bm.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_1b, current_df_bm.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df_bm.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_1b > max_price_index_1b:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df_bm.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                                          prices.append((min_price_index_1b, current_df_bm.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df_bm.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                                    else:
                                         continue                          
                        
                        
                        
                        
                                smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                                larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                                remaining_prices_1c = current_df_bm[(current_df_bm.index > smaller_index_1c) & (current_df_bm.index < larger_index_1c)]
                                remaining_prices_1c_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1c) & (Q_A_Preds_bm.index < larger_index_1c)]
                                remaining_prices_1c_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1c) & (Q_B_Preds_bm.index < larger_index_1c)]

                                if len(remaining_prices_1c) > 1:
                                        max_price_index_1c = remaining_prices_1c_A['Price'].idxmax()
                                        min_price_index_1c = remaining_prices_1c_B['Price'].idxmin()
                                        if current_Q_B_bm.loc[min_price_index_1c, 'Price'] < current_Q_A_bm.loc[max_price_index_1c, 'Price']:
                                            if min_price_index_1c < max_price_index_1c:
                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                charge_level += bottleneck_1
                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                charge_level -= bottleneck_2
                    
                                                profit = (current_df_bm.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                                prices.append((min_price_index_1c, current_df_bm.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df_bm.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                                            elif min_price_index_1c > max_price_index_1c:
                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                  charge_level -= bottleneck_2
                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                  charge_level += bottleneck_1
                    
                                                  profit = (current_df_bm.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                                  prices.append((min_price_index_1c, current_df_bm.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df_bm.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                                            else:
                                                 continue                               
                        
                        
                                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                                        remaining_prices_1d = current_df_bm[(current_df_bm.index > smaller_index_1d) & (current_df_bm.index < larger_index_1d)]
                                        remaining_prices_1d_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1d) & (Q_A_Preds_bm.index < larger_index_1d)]
                                        remaining_prices_1d_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1d) & (Q_B_Preds_bm.index < larger_index_1d)]
    
                                        if len(remaining_prices_1d) > 1:
                                                max_price_index_1d = remaining_prices_1d_A['Price'].idxmax()
                                                min_price_index_1d = remaining_prices_1d_B['Price'].idxmin()
                                                if current_Q_B_bm.loc[min_price_index_1d, 'Price'] < current_Q_A_bm.loc[max_price_index_1d, 'Price']:
                                                    if min_price_index_1d < max_price_index_1d:
                                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                        charge_level += bottleneck_1
                                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                        charge_level -= bottleneck_2
                    
                                                        profit = (current_df_bm.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                                        prices.append((min_price_index_1d, current_df_bm.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df_bm.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                                    elif min_price_index_1d > max_price_index_1d:
                                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                          charge_level -= bottleneck_2
                                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                          charge_level += bottleneck_1
                    
                                                          profit = (current_df_bm.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                                          prices.append((min_price_index_1d, current_df_bm.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df_bm.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                                    else:
                                                         continue  
                                        
                                        
                                        
                                                smaller_index_1e = min(min_price_index_1d, max_price_index_1d)
                                                larger_index_1e = max(min_price_index_1d, max_price_index_1d)
                                                remaining_prices_1e = current_df_bm[(current_df_bm.index > smaller_index_1e) & (current_df_bm.index < larger_index_1e)]
                                                remaining_prices_1e_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1e) & (Q_A_Preds_bm.index < larger_index_1e)]
                                                remaining_prices_1e_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1e) & (Q_B_Preds_bm.index < larger_index_1e)]
    
                                                if len(remaining_prices_1e) > 1:
                                                        max_price_index_1e = remaining_prices_1e_A['Price'].idxmax()
                                                        min_price_index_1e = remaining_prices_1e_B['Price'].idxmin()
                                                        if current_Q_B_bm.loc[min_price_index_1e, 'Price'] < current_Q_A_bm.loc[max_price_index_1e, 'Price']:
                                                            if min_price_index_1e < max_price_index_1e:
                                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                charge_level += bottleneck_1
                                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                charge_level -= bottleneck_2
                    
                                                                profit = (current_df_bm.loc[max_price_index_1e, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1e, 'Price'] * bottleneck_1)/eff_2)
                    
                                                                prices.append((min_price_index_1e, current_df_bm.loc[min_price_index_1e, 'Price'], max_price_index_1e, current_df_bm.loc[max_price_index_1e, 'Price'], profit, charge_level))
                    
                                                            elif min_price_index_1e > max_price_index_1e:
                                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                  charge_level -= bottleneck_2
                                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                  charge_level += bottleneck_1
                    
                                                                  profit = (current_df_bm.loc[max_price_index_1e, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1e, 'Price'] * bottleneck_1)/eff_2)
                    
                                                                  prices.append((min_price_index_1e, current_df_bm.loc[min_price_index_1e, 'Price'], max_price_index_1e, current_df_bm.loc[max_price_index_1e, 'Price'], profit, charge_level))
                                
                                                            else:
                                                                 continue                                          
                                        
                                        
                                        
                                        
                                        
                       
                        
                        
        if len(remaining_prices_2) > 1:
                max_price_index_2 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_2, 'Price'] < current_Q_A_bm.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue  
                        
                        
                      
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_2a = current_df_bm[(current_df_bm.index > smaller_index_2a) & (current_df_bm.index < larger_index_2a)]
                remaining_prices_2a_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2a) & (Q_A_Preds_bm.index < larger_index_2a)]
                remaining_prices_2a_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2a) & (Q_B_Preds_bm.index < larger_index_2a)]

                if len(remaining_prices_2a) > 1:
                        max_price_index_2a = remaining_prices_2a_A['Price'].idxmax()
                        min_price_index_2a = remaining_prices_2a_B['Price'].idxmin()
                        if current_Q_B_bm.loc[min_price_index_2a, 'Price'] < current_Q_A_bm.loc[max_price_index_2a, 'Price']:
                            if min_price_index_2a < max_price_index_2a:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df_bm.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_2a, current_df_bm.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df_bm.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2a > max_price_index_2a:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df_bm.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                                  prices.append((min_price_index_2a, current_df_bm.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df_bm.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                         
                        
                        
                        smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                        larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                        remaining_prices_2b = current_df_bm[(current_df_bm.index > smaller_index_2b) & (current_df_bm.index < larger_index_2b)]
                        remaining_prices_2b_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2b) & (Q_A_Preds_bm.index < larger_index_2b)]
                        remaining_prices_2b_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2b) & (Q_B_Preds_bm.index < larger_index_2b)]

                        if len(remaining_prices_2b) > 1:
                                max_price_index_2b = remaining_prices_2b_A['Price'].idxmax()
                                min_price_index_2b = remaining_prices_2b_B['Price'].idxmin()
                                if current_Q_B_bm.loc[min_price_index_2b, 'Price'] < current_Q_A_bm.loc[max_price_index_2b, 'Price']:
                                    if min_price_index_2b < max_price_index_2b:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df_bm.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_2b, current_df_bm.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df_bm.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2b > max_price_index_2b:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df_bm.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                    
                                          prices.append((min_price_index_2b, current_df_bm.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df_bm.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                
                                    else:
                                         continue                          
                        
                        
                        
                        
                                smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                                larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                                remaining_prices_2c = current_df_bm[(current_df_bm.index > smaller_index_2c) & (current_df_bm.index < larger_index_2c)]
                                remaining_prices_2c_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2c) & (Q_A_Preds_bm.index < larger_index_2c)]
                                remaining_prices_2c_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2c) & (Q_B_Preds_bm.index < larger_index_2c)]

                                if len(remaining_prices_2c) > 1:
                                        max_price_index_2c = remaining_prices_2c_A['Price'].idxmax()
                                        min_price_index_2c = remaining_prices_2c_B['Price'].idxmin()
                                        if current_Q_B_bm.loc[min_price_index_2c, 'Price'] < current_Q_A_bm.loc[max_price_index_2c, 'Price']:
                                            if min_price_index_2c < max_price_index_2c:
                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                charge_level += bottleneck_1
                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                charge_level -= bottleneck_2
                    
                                                profit = (current_df_bm.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                    
                                                prices.append((min_price_index_2c, current_df_bm.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df_bm.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                            elif min_price_index_2c > max_price_index_2c:
                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                  charge_level -= bottleneck_2
                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                  charge_level += bottleneck_1
                    
                                                  profit = (current_df_bm.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                    
                                                  prices.append((min_price_index_2c, current_df_bm.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df_bm.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                
                                            else:
                                                 continue                               
                        
                        
                                        smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                                        larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                                        remaining_prices_2d = current_df_bm[(current_df_bm.index > smaller_index_2d) & (current_df_bm.index < larger_index_2d)]
                                        remaining_prices_2d_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2d) & (Q_A_Preds_bm.index < larger_index_2d)]
                                        remaining_prices_2d_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2d) & (Q_B_Preds_bm.index < larger_index_2d)]
    
                                        if len(remaining_prices_2d) > 1:
                                                max_price_index_2d = remaining_prices_2d_A['Price'].idxmax()
                                                min_price_index_2d = remaining_prices_2d_B['Price'].idxmin()
                                                if current_Q_B_bm.loc[min_price_index_2d, 'Price'] < current_Q_A_bm.loc[max_price_index_2d, 'Price']:
                                                    if min_price_index_2d < max_price_index_2d:
                                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                        charge_level += bottleneck_1
                                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                        charge_level -= bottleneck_2
                    
                                                        profit = (current_df_bm.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                    
                                                        prices.append((min_price_index_2d, current_df_bm.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df_bm.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                                    elif min_price_index_2d > max_price_index_2d:
                                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                          charge_level -= bottleneck_2
                                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                          charge_level += bottleneck_1
                    
                                                          profit = (current_df_bm.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                    
                                                          prices.append((min_price_index_2d, current_df_bm.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df_bm.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                
                                                    else:
                                                         continue  
                                        
                                        
                                        
                                                smaller_index_2e = min(min_price_index_2d, max_price_index_2d)
                                                larger_index_2e = max(min_price_index_2d, max_price_index_2d)
                                                remaining_prices_2e = current_df_bm[(current_df_bm.index > smaller_index_2e) & (current_df_bm.index < larger_index_2e)]
                                                remaining_prices_2e_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2e) & (Q_A_Preds_bm.index < larger_index_2e)]
                                                remaining_prices_2e_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2e) & (Q_B_Preds_bm.index < larger_index_2e)]
    
                                                if len(remaining_prices_2e) > 1:
                                                        max_price_index_2e = remaining_prices_2e_A['Price'].idxmax()
                                                        min_price_index_2e = remaining_prices_2e_B['Price'].idxmin()
                                                        if current_Q_B_bm.loc[min_price_index_2e, 'Price'] < current_Q_A_bm.loc[max_price_index_2e, 'Price']:
                                                            if min_price_index_2e < max_price_index_2e:
                                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                charge_level += bottleneck_1
                                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                charge_level -= bottleneck_2
                    
                                                                profit = (current_df_bm.loc[max_price_index_2e, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2e, 'Price'] * bottleneck_1)/eff_2)
                    
                                                                prices.append((min_price_index_2e, current_df_bm.loc[min_price_index_2e, 'Price'], max_price_index_2e, current_df_bm.loc[max_price_index_2e, 'Price'], profit, charge_level))
                    
                                                            elif min_price_index_2e > max_price_index_2e:
                                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                  charge_level -= bottleneck_2
                                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                  charge_level += bottleneck_1
                    
                                                                  profit = (current_df_bm.loc[max_price_index_2e, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2e, 'Price'] * bottleneck_1)/eff_2)
                    
                                                                  prices.append((min_price_index_2e, current_df_bm.loc[min_price_index_2e, 'Price'], max_price_index_2e, current_df_bm.loc[max_price_index_2e, 'Price'], profit, charge_level))
                                
                                                            else:
                                                                 continue                                         
                                                                      

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_bm_50_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_50_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_10_30=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_30_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_30_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_30_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_50_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_50_70=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_70_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_70_90=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_70_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_70_BM, Q_B_Preds_bm=Q_90_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_30_70=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_70_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_10_90=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_90_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

PF_dam_bm=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Y_r_DAM, Q_B_Preds=Y_r_DAM, Q_A_Preds_bm=Y_r_BM, Q_B_Preds_bm=Y_r_BM,          eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
print(np.round(sum(r_dam_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_dam_bm.iloc[:,4:5].values),2))

[50951.08]
[46628.28]
[52214.41]
[55714.9]
[53486.15]
[45033.84]
[31775.02]
[99396.23]


Perfect foresight only better than single trade at 143829.91, but best perfomerer highest of all 55714.9.